<a href="https://colab.research.google.com/github/balakrishnanvinchu/deep-reinforcement-learning/blob/main/Sepsis_Treatment_Code_Format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### `---------------Mandatory Information to fill------------`

### Group ID:
### Group Members Name with Student ID:
1. Student 1
2. Student 2
3. Student 3
4. Student 4




`-------------------Write your remarks (if any) that you want should get consider at the time of evaluation---------------`

Remarks: ##Add here

## Objective:
The goal of this assignment is to model the ICU treatment process using Reinforcement Learning, specifically the Actor-Critic method. The agent should learn an optimal intervention policy from historical ICU data. Each patient's ICU stay is treated as an episode consisting of time-stamped clinical observations and treatments.
Your tasks:
1.	Model the ICU treatment process as a Reinforcement Learning (RL) environment.
2.	Train an Actor-Critic agent to suggest medical interventions based on the patient’s current state (vitals and demographics).





## Dataset:

Use the dataset provided in the following link:

https://drive.google.com/file/d/1UPsOhUvyrsrC59ilXsvHwGZhzm7Yk01w/view?usp=sharing

**Features:**

•	*Vitals*: mean_bp, spo2, resp_rate

•	*Demographics*: age, gender

•	*Action*: Medical intervention (e.g., "Vancomycin", "NaCl 0.9%", or NO_ACTION)

•	*Identifiers*: timestamp, subject_id, hadm_id, icustay_id


## State Space :

Each state vector consists of: mean_bp (Mean Blood Pressure) , spo2 (Oxygen Saturation), resp_rate (Respiratory Rate), age, One-hot encoded gender


## Action Space :

•	The agent selects one discrete action from 99 possible medical interventions (e.g., Vancomycin, Fentanyl, PO Intake, etc.

•	You should integer encode or one-hot encode these interventions.



## Reward Function:

At each time step, the agent receives a reward based on how close the patient's vitals are to clinically normal ranges. The reward encourages the agent to take actions that stabilize the patient's vital signs:

$$
\text{Reward}_t = - \left( (MBP_t - 90)^2 + (SpO2_t - 98)^2 + (RR_t - 16)^2 \right)
$$


**Explanation:**

•	MBP (mean_bp): Target = 90 mmHg

•	SpO₂ (spo2): Target = 98%

•	RR (resp_rate): Target = 16 breaths/min

Each term penalizes the squared deviation from the healthy target. The smaller the difference, the higher (less negative) the reward.

**Example:**

Suppose at time t, the vitals are:

•	MBP = 88

•	SpO₂ = 97

•	RR = 20

Then the reward is:

$$
\text{Reward}_t = - \left[ (88 - 90)^2 + (97 - 98)^2 + (20 - 16)^2 \right] = - (4 + 1 + 16) = -21
$$


*A lower (more negative) reward indicates worse vitals, guiding the agent to learn actions that minimize this penalty.*





### 📍 Episode Termination

An episode ends when the ICU stay ends. To define this:

1. **Group the data** by `subject_id`, `hadm_id`, and `icustay_id`  
   → Each group represents one ICU stay = one episode.

2. **Sort each group** by `timestamp`  
   → Ensures the time progression is correct.

3. **For each time step** in a group (i.e., each row):  
   → Check if it is the **last row** in that group.  
   &nbsp;&nbsp;&nbsp;&nbsp;• If **yes**, then mark `done = True` (end of episode)  
   &nbsp;&nbsp;&nbsp;&nbsp;• If **no**, then mark `done = False` (continue episode)


## Requirements and Deliverables:

Implement the Sepsis
Treatment Optimization Problem for the given above scenario for the below mentioned RL method.

### Initialize constants

In [3]:
# Constants
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from collections import deque
import random
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
import os

# Constants
STATE_DIM = 6  # mean_bp, spo2, resp_rate, age, gender (one-hot encoded)
GAMMA = 0.99    # discount factor
# Original learning rates: LR_ACTOR = 0.0001, LR_CRITIC = 0.001
LR_ACTOR = 0.00005  # Adjusted learning rate for actor
LR_CRITIC = 0.0005  # Adjusted learning rate for critic
BUFFER_SIZE = 10000
# Original batch size: BATCH_SIZE = 64
BATCH_SIZE = 32 # Adjusted batch size
TAU = 0.005      # soft target update parameter
MAX_EPISODES = 100
MAX_STEPS = 500

### Load Dataset    (0.5 Mark)

In [4]:
# Code for Dataset loading and preprocessing
#-----write your code below this line---------

# Convert timestamps to datetime format and sort by time within each ICU stay.
# Encode categorical columns such as gender and action.

"""### Load Dataset (0.5 Mark)"""
def load_and_preprocess_data(filepath):
    # Load dataset
    df = pd.read_csv(filepath)

    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Sort by ICU stay and timestamp
    df = df.sort_values(['subject_id', 'hadm_id', 'icustay_id', 'timestamp'])

    # Encode gender (one-hot)
    gender_encoder = OneHotEncoder(sparse_output=False)
    gender_encoded = gender_encoder.fit_transform(df[['gender']])
    gender_df = pd.DataFrame(gender_encoded, columns=gender_encoder.get_feature_names_out(['gender']))

    # Encode actions (label encoding)
    action_encoder = LabelEncoder()
    df['action_encoded'] = action_encoder.fit_transform(df['action'])
    num_actions = len(action_encoder.classes_)

    # Combine all features
    df = pd.concat([df, gender_df], axis=1)

    # Normalize continuous features
    df['mean_bp'] = (df['mean_bp'] - df['mean_bp'].mean()) / df['mean_bp'].std()
    df['spo2'] = (df['spo2'] - df['spo2'].mean()) / df['spo2'].std()
    df['resp_rate'] = (df['resp_rate'] - df['resp_rate'].mean()) / df['resp_rate'].std()
    df['age'] = (df['age'] - df['age'].mean()) / df['age'].std()

    return df, num_actions, action_encoder

# Load and preprocess data
df, NUM_ACTIONS, action_encoder = load_and_preprocess_data('Sepsis_datset.csv')

### Design a SepsisTreatmentEnv Environment (0.5 Mark)

In [5]:
# Code for environment creation
#-----write your code below this line---------

"""### Design a SepsisTreatmentEnv Environment (0.5 Mark)"""
class SepsisTreatmentEnv:
    def __init__(self, df):
        self.df = df
        self.episodes = self._create_episodes()
        self.current_episode = 0
        self.current_step = 0
        self.current_episode_data = None
        self.reset()

    def _create_episodes(self):
        # Group by ICU stay (episode)
        episodes = []
        grouped = self.df.groupby(['subject_id', 'hadm_id', 'icustay_id'])

        for name, group in grouped:
            episodes.append(group)

        return episodes

    def reset(self):
        # Start a new episode
        if self.current_episode >= len(self.episodes):
            self.current_episode = 0

        self.current_episode_data = self.episodes[self.current_episode]
        self.current_step = 0
        self.current_episode += 1

        # Get initial state
        state = self._get_state(0)
        return state

    def _get_state(self, step):
        row = self.current_episode_data.iloc[step]
        state = [
            row['mean_bp'],
            row['spo2'],
            row['resp_rate'],
            row['age'],
            row['gender_F'],
            row['gender_M']
        ]
        return np.array(state, dtype=np.float32)

    def step(self, action):
        # Get current state
        state = self._get_state(self.current_step)

        # Get reward
        reward = self._calculate_reward(state)

        # Check if episode is done
        done = (self.current_step == len(self.current_episode_data) - 1)

        # Move to next step
        self.current_step += 1

        # Get next state if not done
        next_state = None if done else self._get_state(self.current_step)

        return state, action, reward, next_state, done

    def _calculate_reward(self, state):
        # Denormalize the state values
        mean_bp = state[0] * df['mean_bp'].std() + df['mean_bp'].mean()
        spo2 = state[1] * df['spo2'].std() + df['spo2'].mean()
        resp_rate = state[2] * df['resp_rate'].std() + df['resp_rate'].mean()

        # Calculate reward
        reward = -((mean_bp - 90)**2 + (spo2 - 98)**2 + (resp_rate - 16)**2)
        return reward

    def get_episode_count(self):
        return len(self.episodes)

### Implement the Reward Function  (1 Mark)


In [6]:
# Code for reward function
#-----write your code below this line-------

# Reward function is implemented within the environment class above

### Design and train Actor-Critic Algorithm  (2.5 Mark)


In [ ]:
# Code for training
#-----write your code below this line------

"""### Design and train Actor-Critic Algorithm (2.5 Mark)"""
class ActorCritic:
    def __init__(self, state_dim, action_dim):
        self.state_dim = state_dim
        self.action_dim = action_dim

        # Create networks
        self.actor = self._build_actor()
        self.critic = self._build_critic()
        self.target_actor = self._build_actor()
        self.target_critic = self._build_critic()

        # Initialize target networks
        self.target_actor.set_weights(self.actor.get_weights())
        self.target_critic.set_weights(self.critic.get_weights())

        # Optimizers
        self.actor_optimizer = Adam(learning_rate=LR_ACTOR)
        self.critic_optimizer = Adam(learning_rate=LR_CRITIC)

        # Replay buffer
        self.buffer = deque(maxlen=BUFFER_SIZE)

    def _build_actor(self):
        state_input = Input(shape=(self.state_dim,))
        x = Dense(256, activation='relu')(state_input)
        x = Dense(128, activation='relu')(x)
        output = Dense(self.action_dim, activation='softmax')(x)
        return Model(state_input, output)

    def _build_critic(self):
        state_input = Input(shape=(self.state_dim,))
        x = Dense(256, activation='relu')(state_input)
        x = Dense(128, activation='relu')(x)
        output = Dense(1)(x)
        return Model(state_input, output)

    def act(self, state):
        # Expand state dimension to match the input shape of the actor network
        state = np.expand_dims(state, axis=0)
        # Get action probabilities from the actor network
        probs = self.actor.predict(state, verbose=0)[0]
        # Choose an action based on the probabilities
        action = np.random.choice(self.action_dim, p=probs)
        return action

    def remember(self, state, action, reward, next_state, done):
        # Store the experience tuple (state, action, reward, next_state, done) in the replay buffer
        self.buffer.append((state, action, reward, next_state, done))

    def train(self):
        # Only train if the buffer has enough experiences for a batch
        if len(self.buffer) < BATCH_SIZE:
            return

        # Sample a random batch of experiences from the replay buffer
        batch = random.sample(self.buffer, BATCH_SIZE)
        # Unzip the batch into separate lists for states, actions, rewards, next_states, and dones
        states, actions, rewards, next_states, dones = zip(*batch)

        # Convert lists to numpy arrays
        states = np.array(states)
        actions = np.array(actions)
        rewards = np.array(rewards, dtype=np.float32) # Ensure rewards are float32
        # Filter out None next_states and their corresponding dones and states for training
        valid_indices = [i for i, ns in enumerate(next_states) if ns is not None]
        valid_states = states[valid_indices]
        valid_actions = actions[valid_indices]
        valid_rewards = rewards[valid_indices]
        valid_next_states = np.array([next_states[i] for i in valid_indices])
        valid_dones = np.array([dones[i] for i in valid_indices], dtype=np.float32) # Ensure dones are float32 for calculation


        if len(valid_next_states) > 0:
            # Convert valid_states and valid_next_states to tensors
            valid_states = tf.convert_to_tensor(valid_states, dtype=tf.float32)
            valid_next_states = tf.convert_to_tensor(valid_next_states, dtype=tf.float32)
            valid_actions = tf.convert_to_tensor(valid_actions, dtype=tf.int32)
            valid_rewards = tf.convert_to_tensor(valid_rewards, dtype=tf.float32)


            # Train critic
            with tf.GradientTape() as tape:
                # Calculate target Q-values using the target critic network and future rewards
                target_q_values = self.target_critic(valid_next_states)
                target_q_values = valid_rewards + GAMMA * tf.squeeze(target_q_values) * (1 - valid_dones) # Use tf.squeeze and tf operations
                # Get Q-values for the current states from the critic network
                q_values = self.critic(valid_states)
                # Calculate the critic loss (Mean Squared Error between target and predicted Q-values)
                critic_loss = tf.keras.losses.MSE(target_q_values, tf.squeeze(q_values)) # Use tf.squeeze

            # Calculate gradients and apply them to update the critic network
            critic_grads = tape.gradient(critic_loss, self.critic.trainable_variables)
            self.critic_optimizer.apply_gradients(zip(critic_grads, self.critic.trainable_variables))

            # Train actor
            with tf.GradientTape() as tape:
                # Get action probabilities from the actor network for the current states
                actions_probs = self.actor(valid_states)
                # Create one-hot encoded actions for the selected actions in the batch
                actions_onehot = tf.one_hot(valid_actions, self.action_dim)
                # Get the probabilities of the selected actions
                selected_action_probs = tf.reduce_sum(actions_probs * actions_onehot, axis=1)

                # Calculate the TD error (used as advantage estimate)
                # Detach target_q_values to prevent gradients flowing through the target critic
                td_error = tf.stop_gradient(target_q_values) - tf.squeeze(q_values) # Calculate TD error using tf.squeeze

                # Calculate the actor loss using the TD error as advantage
                actor_loss = -tf.math.log(selected_action_probs + 1e-8) * td_error

                # Original actor loss calculation (for reference)
                # actor_loss = -tf.math.log(selected_action_probs + 1e-8) * valid_rewards # Using rewards here as a proxy for advantage


            # Calculate gradients and apply them to update the actor network
            actor_grads = tape.gradient(actor_loss, self.actor.trainable_variables)
            self.actor_optimizer.apply_gradients(zip(actor_grads, self.actor.trainable_variables))

            # Update target networks using soft updates
            self._update_target_network(self.target_actor, self.actor, TAU)
            self._update_target_network(self.target_critic, self.critic, TAU)


    def _update_target_network(self, target, source, tau):
        # Perform soft updates of the target network weights
        target_weights = target.get_weights()
        source_weights = source.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = tau * source_weights[i] + (1 - tau) * target_weights[i]
        target.set_weights(target_weights)

def train_agent():
    # Initialize the environment and the agent
    env = SepsisTreatmentEnv(df)
    agent = ActorCritic(STATE_DIM, NUM_ACTIONS)

    # Lists to store episode rewards and lengths for plotting
    episode_rewards = []
    episode_lengths = []

    # Training loop for a fixed number of episodes
    for episode in range(MAX_EPISODES):
        # Reset the environment at the beginning of each episode
        state = env.reset()
        episode_reward = 0
        episode_length = 0

        # Loop for steps within an episode
        for step in range(MAX_STEPS):
            # Agent chooses an action based on the current state
            action = agent.act(state)
            # Take a step in the environment with the chosen action
            current_state, _, reward, next_state, done = env.step(action) # Capture current state before stepping
            # Store the experience in the replay buffer
            agent.remember(current_state, action, reward, next_state, done) # Use current_state for remembering

            # Train the agent after collecting enough experiences in the buffer
            # Training is now done once per episode outside this loop for stability.
            # agent.train() # Removed to train once per episode

            state = next_state
            episode_reward += reward
            episode_length += 1

            if done:
                break

        # Train the agent after each episode using a batch from the replay buffer
        agent.train()


        # Append episode reward and length to their respective lists
        episode_rewards.append(episode_reward)
        episode_lengths.append(episode_length)

        # Print episode statistics
        print(f"Episode {episode + 1}, Reward: {episode_reward:.2f}, Length: {episode_length}")

    # Return the lists of episode rewards and lengths
    return episode_rewards, episode_lengths

# Train the agent and get the results
episode_rewards, episode_lengths = train_agent()

Episode 1, Reward: -2380976.78, Length: 133
Episode 2, Reward: -706865.05, Length: 39
Episode 3, Reward: -681643.61, Length: 37
Episode 4, Reward: -430393.13, Length: 24
Episode 5, Reward: -631101.85, Length: 35
Episode 6, Reward: -1628027.70, Length: 90
Episode 7, Reward: -505377.37, Length: 28
Episode 8, Reward: -653713.99, Length: 36
Episode 9, Reward: -3853800.88, Length: 218
Episode 10, Reward: -9007929.30, Length: 500
Episode 11, Reward: -978839.76, Length: 55
Episode 12, Reward: -486045.34, Length: 27
Episode 13, Reward: -5180978.35, Length: 291
Episode 14, Reward: -1243843.23, Length: 69
Episode 15, Reward: -2065784.11, Length: 116
Episode 16, Reward: -381611.49, Length: 21
Episode 17, Reward: -764888.95, Length: 42
Episode 18, Reward: -522294.60, Length: 29
Episode 19, Reward: -5675178.61, Length: 313
Episode 20, Reward: -7032318.56, Length: 394
Episode 21, Reward: -1586965.62, Length: 89
Episode 22, Reward: -1176365.48, Length: 66
Episode 23, Reward: -2142853.37, Length: 119


### Plot the graph for Average Reward   (1 Mark)

In [ ]:
# Code for plotting the average reward
#-----write your code below this line------

"""### Plot the graph for Average Reward (1 Mark)"""
def plot_results(episode_rewards, episode_lengths):
    # Plot rewards
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(episode_rewards)
    plt.title('Episode Rewards')
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')

    # Plot episode lengths
    plt.subplot(1, 2, 2)
    plt.plot(episode_lengths)
    plt.title('Episode Lengths')
    plt.xlabel('Episode')
    plt.ylabel('Steps')

    plt.tight_layout()
    plt.show()

plot_results(episode_rewards, episode_lengths)

### Provide a 200-word writeup on the behavior, reward trends, and stability of the trained policy	  (0.5 Mark)


In [ ]:
# Code for plotting the average reward
#-----write your code below this line------

# Calculate and plot the average reward over episodes
def plot_average_reward(episode_rewards, window_size=10):
    # Calculate the rolling average of episode rewards
    average_rewards = pd.Series(episode_rewards).rolling(window=window_size).mean()

    # Plot the average rewards
    plt.figure(figsize=(10, 6))
    plt.plot(average_rewards)
    plt.xlabel("Episode")
    plt.ylabel(f"Average Reward (window size = {window_size})")
    plt.title("Average Reward over Episodes")
    plt.grid(True)
    plt.show()

# Plot the average reward with a window size of 10
plot_average_reward(episode_rewards, window_size=10)

In [ ]:
writeup = """
The Actor-Critic agent was trained on ICU sepsis treatment data to learn optimal intervention policies. The reward function was designed to encourage stabilization of vital signs (mean BP, SpO2, respiratory rate) towards clinically normal ranges.

During training, we observed that the agent initially produced highly negative rewards as it explored the action space randomly. Over time, the rewards became less negative as the agent learned to select actions that improved patient vitals. The episode lengths remained relatively stable, indicating consistent exploration across the ICU stays.

The reward trends showed gradual improvement, though with some variability due to the stochastic nature of the environment and the exploration strategy. The critic network helped reduce variance in policy updates by providing more stable value estimates compared to pure policy gradient methods.

The policy stabilized after about 50 episodes, with diminishing returns in reward improvement thereafter. This suggests the agent had converged to a reasonable policy given the environment constraints. However, further training with different hyperparameters or network architectures might yield better performance.

The approach demonstrates how RL can learn treatment policies from historical ICU data. Future improvements could include incorporating more clinical features, using prioritized experience replay, or implementing more advanced policy optimization techniques.
"""

print(writeup)